In this problem, you need to train a Hopfield network using the attached image. Use the attached training image to train your network and then evaluate its performance employing the attached test image. To this end, you need to complete each cell step by step.

You have to submit the final saved image as well as the completed notebook. 


In [1]:
import numpy as np
import random
import PIL
from PIL import Image
import os, time, threading
import re

In [2]:
#Convert a matrix to a vector
def mat2vec(x):
    """convert the matrix x to a vector
    input: 
        [[1,2,3],
        [4,5,6],
        [7,8,9]]
    output:
        [1,2,3,4,5,6,7,8,9]"""

    #### Implement your code ###
    tmp1 = x.flatten()
    return tmp1

In [3]:
def create_W(x):
  """  
  Create a square matrix with the same size as the input size.
  Note 1: The weight matrix must be symmetric
  Tip 1: For row i and column j, while i != j, place the value x [i] * x [j] in the weight matrix w [i, j]
  Tip 2: For row i and column j, while i=j, put the value 0 in the weight matrix w [i, j]
  """
  #### Impelement your code ### 
  s = time.time()
  n = x.shape[0]
  w = np.zeros((n,n))
  c = 0
  for i in range(n):
    for j in range(n):
      if i == j:
        continue
      w[i, j] = x[i]*x[j]  
      c += 1
      if c%5000000==0:
        print(c)
  return w, time.time()-s

In [4]:
#Read an image file and convert it to a pattern of the image
def readImg2array(file,size, threshold= 145):
    img = Image.open(file).convert(mode="L")
    img= img.resize(size)
    #img.thumbnail(size,Image.ANTIALIAS)
    imgArray = np.asarray(img,dtype=np.uint8)
    x = np.zeros(imgArray.shape,dtype=np.float64)

    """
    Set the value to 1 for each pixel value with the larger than the threshold,
    and Set the value to -1 for each image pixel with a value of 0.
    """
    #### Implement your code ###
    x[imgArray>threshold] = 1
    x[imgArray<=threshold] = -1

    return x

In [5]:
#Convert a numpy array to an image file like Jpeg
def array2img(data, outFile = None):

    #data is 1 or -1 matrix
    y = np.zeros(data.shape,dtype=np.uint8)
    y[data==1] = 255
    y[data==-1] = 0
    img = Image.fromarray(y,mode="L")
    if outFile is not None:
        img.save(outFile)
    return img

In [6]:
#Update the test input pattern (y_vec) based on the weight matrix
def update(w,y_vec,theta=0.5,max_iter=100):

    """
    Once in a while, select a random number between 0 and the size of the input vector -1.
    Then use the random number line of the weight matrix to multiply internally by the input vector.
    Subtract the result from theta
    If the end result is greater than 0, enter a value of 1 in the input vector, otherwise replace -1.
    """
    #### Impelement your code ###
    s = time.time()
    iter = 0
    while True:
        i = np.random.randint(0, w.shape[0])
        wi = w[i,:]
        si = wi @ y_vec.T
        y_prev = y_vec.copy()
        y_vec[i] = 1 if si>theta else -1
        print(f'--- iter({iter}) ---')
        print(f'prev: {y_prev}')
        print(f'new: {y_vec}')
        iter += 1
        if np.all(y_prev==y_vec) and iter >= max_iter:
            break
        
    return y_vec, time.time()-s

In [7]:
#Update the test input pattern (y_vec) based on the weight matrix
def update_sync(w,y_vec,theta=0.5,max_iter=100):

    """
    Once in a while, select a random number between 0 and the size of the input vector -1.
    Then use the random number line of the weight matrix to multiply internally by the input vector.
    Subtract the result from theta
    If the end result is greater than 0, enter a value of 1 in the input vector, otherwise replace -1.
    """
    #### Impelement your code ###
    s = time.time()
    n = y_vec.shape[0]
    iter = 0
    while True:
        y_new = np.zeros_like(y_vec)
        for i in range(n):
            wi = w[i,:]
            si = wi @ y_vec.T
            y_new[i] = 1 if si > theta else -1
            
        y_prev = y_vec.copy()
        y_vec = y_new.copy()
        
        print(f'--- iter({iter}) ---')
        print(f'prev: {y_prev}')
        print(f'new: {y_vec}')
        iter += 1
        if np.all(y_prev==y_vec) and iter >= max_iter:
            break
        
    return y_vec, time.time()-s

In [8]:

def hopfield(train_file, test_file,theta=0.5, max_iter=1000, size=(100,100),threshold=60):

   """
   Using the built-in functions, Do the following steps:
   1- Read the input image and extract its pattern
   2. Convert the obtained pattern into a vector
   3- Make a weight matrix based on the vector of the previous step
   4- Read the test image and extract its pattern
   5- Convert the test pattern into a vector and give it as the input of the update function along with the built-in weight matrix.

   """
#### Impelement your code ###
   print('Loading Train & Test Image...')
   x_train = readImg2array(train_file, size, threshold)
   array2img(x_train, 'train_binary.jpg')
   x_test = readImg2array(test_file, size, threshold)
   array2img(x_test, 'test_binary.jpg')
   
   print('Training Weights based on Train Sample...')
   w, triantime = create_W(mat2vec(x_train))
   print(f'Took: {triantime} seconds')
   
   y_vec = mat2vec(x_test).copy()
   oshape = size
   print('Updating Image through Network...')
   y_vec_after, runtime = update(w=w,y_vec=y_vec,theta=theta,max_iter=max_iter)
   print(f'Took: {runtime}')
   y_vec_after = y_vec_after.reshape(oshape)
   after_img = array2img(y_vec_after,outFile=None)
   after_img.save("result.jpg")
   after_img.show()
        

In [9]:
train_file = 'train.jpg'
test_file =  'test.jpg'

hopfield(train_file, test_file, theta=0.5,max_iter=30000,size=(100,100),threshold=60)

Loading Train & Test Image...
Training Weights based on Train Sample...
5000000
10000000
15000000
20000000
25000000
30000000
35000000
40000000
45000000
50000000
55000000
60000000
65000000
70000000
75000000
80000000
85000000
90000000
95000000
Took: 60.85017967224121 seconds
Updating Image through Network...
--- iter(0) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(1) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(2) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(3) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(4) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(5) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(6) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(7) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(8) ---
prev: [1. 1. 1. ... 1. 1. 1.]
new: [1. 1. 1. ... 1. 1. 1.]
--- iter(9) ---
p